In [ ]:
include("../load.jl")

In [ ]:
include("../../data/speed_reducer.jl")

In [ ]:
gm = sagemark_to_GlobalModel(3, false)
# gm = speed_reducer()
set_optimizer(gm, CPLEX_SILENT)
set_param(gm, :abstol, 1e-7)
set_param(gm, :ignore_accuracy, true)

# Actually trying to optimize...
find_bounds!(gm, all_bounds=true)
uniform_sample_and_eval!(gm)

surveysolve(gm)
println("Approximation accuracies: ", evaluate_accuracy(gm))

In [ ]:
clear_tree_constraints!(gm)
add_tree_constraints!(gm)
@time optimize!(gm)

In [ ]:
# Solving of model
set_param(gm, :ignore_accuracy, true)
# add_tree_constraints!(gm)
# optimize!(gm)
vals = solution(gm);
add_infeasibility_cuts!(gm)
optimize!(gm)
while abs(gm.cost[end] - gm.cost[end-1]) > get_param(gm, :abstol)
    add_infeasibility_cuts!(gm)
    optimize!(gm)
end
println("X values: ", vals)
println("Optimal X: ", vcat(exp.([5.01063529, 3.40119660, -0.48450710]), [-147-2/3]))

In [ ]:
gm.bbls[1].learners[end]
# gm.bbls[1].actuals
# gm.bbls[1].mi_constraints

In [ ]:
IAI.show_in_browser(gm.bbls[2].learners[end])

In [ ]:
bbr = gm.bbls[1]
score = IAI.score(bbr.learners[end], bbr.X, bbr.Y)

In [ ]:
gm.model

In [ ]:
# Printing numbers of variables and constraints
println("Variables: $(length(JuMP.all_variables(gm.model)))")
println("Constraints: $(sum(length(all_constraints(gm.model, type[1], type[2])) for type in JuMP.list_of_constraint_types(gm.model))
)")

In [ ]:
using Plots
using LatexStrings
# First the constraint
bbc = gm.bbls[2]
feas_idxs = findall(x -> x .>= 0, bbc.Y)
infeas_idxs = findall(x -> x .< 0, bbc.Y)
scatter(bbc.X[feas_idxs,1],
bbc.X[feas_idxs, 2], bbc.X[feas_idxs, 3], 
        legend = false, label = "feasible", color = :green,  fmt = :png)
plt = scatter!(bbc.X[infeas_idxs,1],
bbc.X[infeas_idxs, 2], bbc.X[infeas_idxs, 3], 
        title = "Constraint samples",
xlabel = L"x_1", ylabel = L"x_2", zlabel = L"x_3", 
legend = true, label = "infeasible", color = :red,  fmt = :png)
savefig(plt, "constr")

In [ ]:
bbr = gm.bbls[1]
plt = scatter(bbr.X[:,1],
bbr.X[:, 2], bbr.Y, 
        title = "Objective samples",
xlabel = L"x_1", ylabel = L"x_2", zlabel = L"f(\mathbf{x})", 
legend = false, color = :green,  fmt = :png)
savefig(plt, "obj")